In [37]:
import pickle

with open('../model/data/paradeigma_ts_data_preprocessing.pkl', 'rb') as f:
    target_data_frames = pickle.load(f)

In [38]:
num_sessions = 40

In [39]:
missing_eda_count = 0
missing_eda_session = []
missing_eda_coord = []
for session in range(num_sessions):
    if target_data_frames[session+1]['eda'].isna().sum() != 0:
        missing_eda_count += target_data_frames[session+1]['eda'].isna().sum()
        missing_eda_session.append(session+1)
        
missing_temp_count = 0
missing_temp_session = []
for session in range(num_sessions):
    if target_data_frames[session+1]['temp'].isna().sum() != 0:
        missing_temp_count += target_data_frames[session+1]['temp'].isna().sum()
        missing_temp_session.append(session+1)
        
missing_ibi_count = 0
missing_ibi_session = []
for session in range(num_sessions):
    if target_data_frames[session+1]['ibi'].isna().sum() != 0:
        missing_ibi_count += target_data_frames[session+1]['ibi'].isna().sum()
        missing_ibi_session.append(session+1)
    

In [40]:
missing_eda_count = 0
missing_eda_session = []
missing_eda_coord = []
for session in range(num_sessions):
    if target_data_frames[session+1]['eda'].isna().sum() != 0:
        missing_eda_count += target_data_frames[session+1]['eda'].isna().sum()
        missing_eda_session.append(session+1)
        
missing_temp_count = 0
missing_temp_session = []
for session in range(num_sessions):
    if target_data_frames[session+1]['temp'].isna().sum() != 0:
        missing_temp_count += target_data_frames[session+1]['temp'].isna().sum()
        missing_temp_session.append(session+1)
        
missing_ibi_count = 0
missing_ibi_session = []
for session in range(num_sessions):
    if target_data_frames[session+1]['ibi'].isna().sum() != 0:
        missing_ibi_count += target_data_frames[session+1]['ibi'].isna().sum()
        missing_ibi_session.append(session+1)
    

In [41]:
print("Session of Missing EDA Datas: ", missing_eda_session)
print("Numbers of Missing EDA Datas: ", missing_eda_count)
print("-"*30)
print("Session of Missing Temp Datas: ", missing_temp_session)
print("Numbers of Missing Temp Datas: ", missing_temp_count)
print("-"*30)
print("Session of Missing IBI Datas: ", missing_ibi_session)
print("Numbers of Missing IBI Datas: ", missing_ibi_count)
print("-"*30)

Session of Missing EDA Datas:  [12, 17, 18, 36, 38, 39]
Numbers of Missing EDA Datas:  747
------------------------------
Session of Missing Temp Datas:  [12, 17, 18, 36, 38, 39]
Numbers of Missing Temp Datas:  747
------------------------------
Session of Missing IBI Datas:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
Numbers of Missing IBI Datas:  3875
------------------------------


In [42]:
missing_eda_dict = {}
for missing_session in missing_eda_session:
    missing_index = target_data_frames[missing_session]['eda'][target_data_frames[missing_session]['eda'].isna()].index
    missing_eda_dict[missing_session] = missing_index
    
missing_temp_dict = {}
for missing_session in missing_temp_session:
    missing_index = target_data_frames[missing_session]['temp'][target_data_frames[missing_session]['temp'].isna()].index
    missing_temp_dict[missing_session] = missing_index
    
missing_ibi_dict = {}
for missing_session in missing_ibi_session:
    missing_index = target_data_frames[missing_session]['eda'][target_data_frames[missing_session]['eda'].isna()].index
    missing_ibi_dict[missing_session] = missing_index

In [43]:
from tqdm import tqdm

for session in tqdm(range(num_sessions)):
    for segment in range(len(target_data_frames[session+1])):
        try: target_data_frames[session+1] = target_data_frames[session+1].drop(['ibi'], axis=1)
        except: pass
        target_data_frames[session+1] = target_data_frames[session+1].dropna()
        target_data_frames[session+1] = target_data_frames[session+1].reset_index(drop=True)

100%|██████████| 40/40 [00:14<00:00,  2.77it/s]


In [44]:
def sequence_padding(ts_list, padding_length = 50, mode = 'constant'):
    
    padding_value=0
    
    if (type(ts_list) != type([])) :
        ts_list = [padding_value] * padding_length
    
    elif len(ts_list) >= padding_length :
        ts_list = ts_list[0:padding_length]
    
    elif mode == 'constant':
        length = padding_length - len(ts_list)
        extend_list = [padding_value] * length
        ts_list = ts_list + extend_list    
        
    elif mode == 'replicate':
        
        quotient = padding_length // len(ts_list)
        remainder = padding_length % len(ts_list)
        result = ts_list * quotient
        result += ts_list[:remainder]
        ts_list = result
        
    return ts_list            

In [45]:
import copy
padded_target_data_frames = copy.deepcopy(target_data_frames)

for session in tqdm(range(num_sessions)):
    padded_target_data_frames[session+1]['eda'] = padded_target_data_frames[session+1]['eda'].apply(sequence_padding, padding_length = 50, mode = 'replicate')
    padded_target_data_frames[session+1]['temp'] = padded_target_data_frames[session+1]['temp'].apply(sequence_padding, padding_length = 50, mode = 'replicate')

100%|██████████| 40/40 [00:00<00:00, 469.10it/s]


In [46]:
for session in range(num_sessions):
    index = []
    for segment in range(len(padded_target_data_frames[session + 1])):
        if 'neutral' in padded_target_data_frames[session+1]['emotion'][segment]:
            if padded_target_data_frames[session+1]['emotion'][segment].replace('neutral', '')
        
        if padded_target_data_frames[session+1]['emotion'][segment].count(';') == 0:
            index.append(segment)
    padded_target_data_frames[session+1] = padded_target_data_frames[session+1].iloc[index].reset_index(drop=True)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 225, 226, 227, 228, 229, 230

In [66]:
for session in range(num_sessions):
    for segment in range(len(padded_target_data_frames[session+1])):
        padded_target_data_frames[session+1] = padded_target_data_frames[session+1][
            (padded_target_data_frames[session+1]['emotion'] != 'neutral') & 
            (padded_target_data_frames[session+1]['emotion'] != 'happy')
            ].reset_index(drop=True)

In [62]:
import torch
import sklearn.svm as svm
import sklearn.metrics as mt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler

/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [67]:
train_eda = []
train_temp = []
target = []

for session in range(num_sessions):
    for segment in range(len(padded_target_data_frames[session+1])):
        train_eda.append(padded_target_data_frames[session+1]['eda'][segment])
        train_temp.append(padded_target_data_frames[session+1]['temp'][segment])
        target.append(padded_target_data_frames[session+1]['emotion'][segment])

In [68]:
len(train_eda), len(train_temp), len(target)

(503, 503, 503)

In [69]:
target_V = []
target_A = []
for session in range(num_sessions):
    for segment in range(len(padded_target_data_frames[session + 1])):
        target_V.append(float(padded_target_data_frames[session+1]['valence'][segment]))
        target_A.append(float(padded_target_data_frames[session+1]['arousal'][segment]))

In [70]:
len(target_V), len(target_A)

(503, 503)

In [71]:
unique_emotion_target = list(set(target))
len(unique_emotion_target)

5

In [72]:
import pandas as pd

In [73]:
target_VA_df = pd.DataFrame([target_V, target_A]).T
target_VA_df.columns = ['Vanlence', 'Arousal']
target_VA_df

,Vanlence,Arousal
0,3.9,4.2
1,3.0,3.2
2,2.2,3.6
3,2.9,3.3
4,3.0,2.8
...,...,...
498,2.6,2.9
499,2.8,2.8
500,2.9,3.6
501,3.1,3.7


In [74]:
# Emotion Encoding

encoded_target = []
for i in range(len(target)):
    for j in range(len(unique_emotion_target)):
        if target[i] == unique_emotion_target[j]:
            encoded_target.append(j)            

In [75]:
scaler = MinMaxScaler()
scaled_target_VA = scaler.fit_transform(target_VA_df)
scaled_target_VA_df = pd.DataFrame(scaled_target_VA)
scaled_target_VA_df.columns = ['Vanlence', 'Arousal']
scaled_target_VA_df['Vanlence']

0      0.884615
1      0.538462
2      0.230769
3      0.500000
4      0.538462
         ...   
498    0.384615
499    0.461538
500    0.500000
501    0.576923
502    0.423077
Name: Vanlence, Length: 503, dtype: float64

In [76]:
train = torch.concat([torch.tensor(train_eda), torch.tensor(train_temp)], dim=-1)
train.shape

torch.Size([503, 100])

In [77]:
svm_clf = svm.SVC(kernel = 'linear')
X_train, X_test, y_train, y_test = train_test_split(train, encoded_target, test_size = 0.2, random_state=42)
svm_clf.fit(X_train, y_train)
svm_clf_predictions = svm_clf.predict(X_test)
print("Accuracy: {}%".format(svm_clf.score(X_test, y_test) * 100 ))
print("f1_score: {}%".format(f1_score(y_test, svm_clf_predictions, average='weighted')*100))
print(classification_report(svm_clf_predictions, y_test))

Accuracy: 24.752475247524753%
f1_score: 11.89005997373931%
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         0
           2       0.92      0.24      0.39        98
           3       0.03      1.00      0.06         1
           4       0.00      0.00      0.00         0

    accuracy                           0.25       101
   macro avg       0.19      0.25      0.09       101
weighted avg       0.90      0.25      0.38       101



/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [78]:
svm_clf = svm.SVC(kernel = 'rbf', C = 10.0, gamma=0.1)
X_train, X_test, y_train, y_test = train_test_split(train, encoded_target, test_size = 0.2, random_state=42)
svm_clf.fit(X_train, y_train)
svm_clf_predictions = svm_clf.predict(X_test)
print("Accuracy: {}%".format(svm_clf.score(X_test, y_test) * 100 ))
print("f1_score: {}%".format(f1_score(y_test, svm_clf_predictions, average='weighted')*100))
print(classification_report(svm_clf_predictions, y_test))

Accuracy: 39.603960396039604%
f1_score: 36.94215189094232%
              precision    recall  f1-score   support

           0       0.27      0.32      0.29        19
           1       0.80      0.40      0.53        10
           2       0.54      0.33      0.41        42
           3       0.46      0.55      0.50        29
           4       0.00      0.00      0.00         1

    accuracy                           0.40       101
   macro avg       0.41      0.32      0.35       101
weighted avg       0.49      0.40      0.42       101



In [79]:
gnb_clf = GaussianNB()
X_train, X_test, y_train, y_test = train_test_split(train, encoded_target, test_size = 0.2, random_state=32)
gnb_clf.fit(X_train, y_train)
gnb_clf_prediction = gnb_clf.predict(X_test)
print("Accuracy: {}%".format(gnb_clf.score(X_test, y_test)))
print("f1_score: {}%".format(f1_score(y_test, gnb_clf_prediction, average='weighted')))
print(classification_report(y_test, gnb_clf_prediction))

Accuracy: 0.1485148514851485%
f1_score: 0.08322370698608322%
              precision    recall  f1-score   support

           0       0.28      0.33      0.30        15
           1       0.14      0.75      0.23        12
           2       0.00      0.00      0.00        32
           3       0.00      0.00      0.00        30
           4       0.10      0.08      0.09        12

    accuracy                           0.15       101
   macro avg       0.10      0.23      0.12       101
weighted avg       0.07      0.15      0.08       101



/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [80]:
ran_clf = RandomForestClassifier(n_estimators=64, max_depth=32,random_state=0)
X_train, X_test, y_train, y_test = train_test_split(train, encoded_target, test_size = 0.2, random_state=42)
ran_clf.fit(X_train,y_train)
ran_clf_prediction = ran_clf.predict(X_test)
print("Accuracy: {}%".format(ran_clf.score(X_test, y_test)))
print("f1_score: {}%".format(f1_score(y_test, ran_clf_prediction, average='weighted')))
print(classification_report(y_test, ran_clf_prediction))

Accuracy: 0.32673267326732675%
f1_score: 0.3241169601910024%
              precision    recall  f1-score   support

           0       0.38      0.41      0.39        22
           1       0.50      0.40      0.44         5
           2       0.26      0.35      0.30        26
           3       0.40      0.34      0.37        35
           4       0.11      0.08      0.09        13

    accuracy                           0.33       101
   macro avg       0.33      0.32      0.32       101
weighted avg       0.33      0.33      0.32       101



In [81]:
tree_clf = DecisionTreeClassifier()  
X_train, X_test, y_train, y_test = train_test_split(train, encoded_target, test_size = 0.2, random_state=42)
tree_clf.fit(X_train,y_train)
tree_clf_prediction = tree_clf.predict(X_test)
print("Accuracy: {}%".format(tree_clf.score(X_test, y_test)))
print("f1_score: {}%".format(f1_score(y_test, tree_clf_prediction, average='weighted')))
print(classification_report(y_test, tree_clf_prediction))

Accuracy: 0.31683168316831684%
f1_score: 0.3250847858323919%
              precision    recall  f1-score   support

           0       0.20      0.27      0.23        22
           1       0.00      0.00      0.00         5
           2       0.33      0.35      0.34        26
           3       0.52      0.40      0.45        35
           4       0.25      0.23      0.24        13

    accuracy                           0.32       101
   macro avg       0.26      0.25      0.25       101
weighted avg       0.34      0.32      0.33       101

